## Jacob Vander Sanden
## Web Scraping Project - Part 1: Scraping
## 11/13/2024

In [2]:
# imports 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By # used to import different ways to access data in the XML or HTML file
from selenium.webdriver.chrome.service import Service # no longer need to download a driver file, use service
from webdriver_manager.chrome import ChromeDriverManager # used to manage the Chrome driver to emulate a Chrome web browser
import time
import random

In [3]:
# import excel with amazon links to loop through 

best_selling_books = pd.read_csv('best_selling_books.csv')

urls = best_selling_books['Amazon Link']

In [4]:
# Function to scroll from the top to the bottom of the web page
def random_scroll(browser, total_wait_time):
   
    total_height = browser.execute_script("return document.body.scrollHeight")
   
    scroll_steps = random.randint(3, 10)  
    
    scroll_increment = total_height // scroll_steps
   
    time_per_step = total_wait_time / scroll_steps

    for step in range(scroll_steps):
       
        browser.execute_script(f"window.scrollBy(0, {scroll_increment});")
        
        random_wait = random.uniform(0.5 * time_per_step, 1.5 * time_per_step)
       
        time.sleep(random_wait)

    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Initialize the Selenium web driver (Chrome in this case)
browser = webdriver.Chrome()

# Create empty lists to store the extracted data
amazon_average_rating = []
amazon_sales_rank = []
amazon_total_reviews = []

browser.maximize_window()

# choose what urls to scrape 
for i, url in enumerate(urls[0:174], start=1):
    browser.get(url)

    # Add a random delay before scraping
    total_wait_time = random.uniform(2, 20)
    random_scroll(browser, total_wait_time)

    # Scrape the rating
    rating_element = browser.find_element(By.XPATH, "//span[@data-hook='rating-out-of-text']")
    rating_text = rating_element.text
    amazon_average_rating.append(rating_text)
    
    # Scrape the rank and account for borh different paths
    rank_elements = browser.find_element(By.XPATH,
    ("//span[contains(@class, 'a-list-item') and contains(., 'Best Sellers Rank')] | "
     "//th[contains(text(), 'Best Sellers Rank')]/following-sibling::td"))
    rank_text = rank_elements.text
    amazon_sales_rank.append(rank_text)
    
    # Scrape the total reviews
    total_reviews_element = browser.find_element(By.XPATH, "//span[@id='acrCustomerReviewText']")
    total_reviews_text = total_reviews_element.text
    amazon_total_reviews.append(total_reviews_text)

    # Print page progress
    print(f"page {i} done")

# Close the automated browser
browser.quit()

# Verify the data by printing the lengths of each list
print(f"Ratings Scraped: {len(amazon_average_rating)}")
print(f"Sales Ranks Scraped: {len(amazon_sales_rank)}")
print(f"Reviews Scraped: {len(amazon_total_reviews)}")

page 1 done


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x00007FF79FD26CB5+28821]
	(No symbol) [0x00007FF79FC93840]
	(No symbol) [0x00007FF79FB3578A]
	(No symbol) [0x00007FF79FB0F4F5]
	(No symbol) [0x00007FF79FBB6247]
	(No symbol) [0x00007FF79FBCECE2]
	(No symbol) [0x00007FF79FBAF0A3]
	(No symbol) [0x00007FF79FB7A778]
	(No symbol) [0x00007FF79FB7B8E1]
	GetHandleVerifier [0x00007FF7A005FCAD+3408013]
	GetHandleVerifier [0x00007FF7A007741F+3504127]
	GetHandleVerifier [0x00007FF7A006B5FD+3455453]
	GetHandleVerifier [0x00007FF79FDEBDBB+835995]
	(No symbol) [0x00007FF79FC9EB5F]
	(No symbol) [0x00007FF79FC9A814]
	(No symbol) [0x00007FF79FC9A9AD]
	(No symbol) [0x00007FF79FC8A199]
	BaseThreadInitThunk [0x00007FFFAD1E259D+29]
	RtlUserThreadStart [0x00007FFFAF3AAF38+40]


In [ ]:
# Create DataFrame and export data
amazon_raw = pd.DataFrame({
    'Amazon_Rating':amazon_average_rating,
    'Amazon_Sales_Rank': amazon_sales_rank,
    'Amazon_Total_Reviews': amazon_total_reviews
})

print(amazon_raw.shape)
amazon_raw.to_csv('amazon_raw.csv', index=False, encoding='utf-8')
